Todo:
- pohlídat oscrapování odkazů nahoře (řečník: Minulý) / vyřešeno, ale teď se ztrácí "(pokračuje Tomio Okamura)" zde: (pokračuje Tomio Okamura) – protože to není v odstavci

- obrácené vnoření A a B v 1998ps_stenprot_003schuz_s003023.htm
- "(pokračuje" v 1920ns_se_182schuz_s182005
- procedurální věci v 1945pns_stenprot_003schuz_s003001.tm
-  Federální shromáždění ČSFR, 1992 – 1992/ 
Sněmovna liud nemá ani tučné, ani odkazů

In [15]:
import os
import csv
import json
import random
from bs4 import BeautifulSoup
import concurrent.futures

In [16]:
kam = "data_raw/schuze"

try:
    os.makedirs(kam)
except:
    pass

In [17]:
import re

In [18]:
# re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", "zxz").group()

In [19]:
mesice = {
    'ledna':1,
    'února':2,
    'března':3,
    'dubna':4,
    'května':5,
    'června':6,
    'července':7,
    'srpna':8,
    'září':9,
    'října':10,
    'listopadu':11,
    'prosince':12
}

funkce = [
    'předseda',
    'předsedkyně',
    'předs.',
    'místopředseda',
    'místopředsedkyně',
    'podpředseda',
    'podpředsedkyně'
    'poslanec',
    'poslankyně',
    'posl.',
    'ministr',
    'ministryně',
    'zpravodaj',
    'zpravodajka',
    'zástupce',
    'zástupkyně',
    'sen.',
    'senátor',
    'senátorka',
    'společná',
    'společná',
    'tajemník',
    'tajemnice',
    'ministerský',
    'ministerská',
    'guvernér'
]

In [20]:
def oscrapovani_stenozaznamu(soubor, zdroj=False):
    
    def ziskej_metadata(soubor):
        if "psse" not in soubor:
            komora_kratce = soubor.split("_")[0][4:]
        else:
            komora_kratce = "psse"
        obdobi = soubor.split("_")[0][0:4]
        schuze = soubor.split("_")[-1][1:4]
        cast = soubor.split("_")[-1][-7:-4]

        return komora_kratce, int(obdobi), int(schuze), int(cast.removesuffix("."))

    def najdi_datum(polivka):
        

        surove_datum = ""
        if polivka.find(class_="date"):
            surove_datum = polivka.find(class_="date").text.replace('\xa0',' ').lower()
        if surove_datum == "":
            try:
                surove_datum = re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", polivka.text).group()
            except:
                surove_datum = polivka.find("a", href="../index.htm").text.lower()

        try:
            rok = int(surove_datum.split(" ")[-1])
            mesic = surove_datum.split(" ")[-2]        
            den = int(surove_datum.split(" ")[-3].removesuffix('.'))
            return f"{rok}-{mesice[mesic]:02}-{den:02}"
        except:
            return None

    def najdi_komoru_a_schuzi(polivka):
        odkazy = polivka.find_all('a')
        komora, schuze = None, None
        try:
            komora = soup.find("h1").text.strip()
        except:
            komora = soup.find(id="breadcrumb").find_all("a")[1].text.strip()
        finally:
            pass
        for a in odkazy:
            if 'href' in a.attrs:
                if './index.htm' in a['href']:
                    schuze = a.text.strip()
        return komora, schuze

    def ciste_odstavce(seznam):
        return ' '.join(seznam).replace('\xa0',' ').replace('***','').removeprefix(':').strip()
    
    metadata = ziskej_metadata(soubor)

    slovnik_metadata = {
        'komora': metadata[0],
        'obdobi': metadata[1],
        'schuze': metadata[2],
        'cast': metadata[3],
        'soubor': soubor
    }

    if "1996ps" in soubor:
        parsovatko = "html5lib" # kvůli 1996ps_stenprot_016schuz_s016005.htm aj., odkazy začínají mimo odstavce
    else:
        parsovatko = "lxml"
    
    with open(os.path.join("downloads/schuze/",soubor), "r", encoding="windows-1250") as schuze:
        schuze = schuze.read().replace('\n',' ')
        soup = BeautifulSoup(schuze, parsovatko)

    slovnik_metadata['datum'] = najdi_datum(soup)
    slovnik_metadata['komora_komplet'], slovnik_metadata['schuze_komplet'] = najdi_komoru_a_schuzi(soup)

    if "NS RČS" in soup.find(id="breadcrumb").text:
        if "Senát" in [a.text for a in soup.find(id="breadcrumb").find_all('a')]:
            slovnik_metadata['komora_komplet'] += ", Senát"
            slovnik_metadata['komora'] = "se"
        elif "Poslanecká sněmovna" in [a.text for a in soup.find(id="breadcrumb").find_all('a')]:
            slovnik_metadata['komora_komplet'] += ", Poslanecká sněmovna"
            slovnik_metadata['komora'] = "ps"
    
    slovnik_metadata['prepsano'] = True
    slovnik_metadata['autorizovano'] = True

    orez = 130
    
    projevy = []
    
    if "nebyl dosud přepsán. Prosím, zkuste později." in soup.text:
        slovnik_metadata['prepsano'] = False
        return slovnik_metadata, projevy
        
    if ("Neprošlo jazykovou korekturou, neautorizováno" in soup.text) or ("Neprošlo opravou po digitalizaci" in soup.text):
        slovnik_metadata['autorizovano'] = False
    
    uz_byl_tucnej = False
    poradi = 1
    projev = {'mluvci':None,'poradi':poradi,'soubor':soubor,'mluvci_id':None}
    odstavce = []

    vsechny_odstavce = soup.find_all("p")

    zacatek_dokumentu = 5
    for index, p in enumerate(vsechny_odstavce[0:7]):
        if ("Minulý" in p.text) and ("Dolů" in p.text) and ("Další" in p.text) and (len(p.text.strip()) < 60):
            zacatek_dokumentu = index + 1
    
    for p in vsechny_odstavce[zacatek_dokumentu:-10]:
        if zdroj == True:
            print("____" + p.text[0:orez])

        if ("1992fs_sl" in soubor) or ("1990fs_sl" in soubor):

            if (':' in p.text.strip()[0:orez - 30]) and (len([l for l in p.text.strip().split(":")[0] if l.isupper()]) > 1):
                
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                uz_byl_tucnej = True
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.text.split(':')[0].strip()
                projev['mluvci_id'] = None
                odstavce.append(' '.join(p.text.split(':')[1:]).strip())

            else:
                odstavce.append(p.text.strip())

        else:

            if zdroj:
                if p.find("b"):
                    print("TUČNÉ")

            if ':' in p.text.strip()[0:orez] and (
                    p.find('b', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False) 
                    or 
                    p.find('u', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False)
                ) and (
                    "výsledkem:" not in p.text # kvůli 2006se_stenprot_015schuz_s015018.htm
                ):

                pocitadlo_zvyrazneni = 0
                prvni_tucne_je_druhe_tucne = False
                zadna_zminka_o_funkci = True
                
                if len(p.find_all("b")) > 1: # kvůli 1920ns_se_stenprot_195schuz_s195001.htm
                    if zdroj == True:
                        print("počítám tučnost")
                    for tucne in p.find_all("b"):
                        if tucne.text.strip() in p.text.strip()[0:orez]:
                            pocitadlo_zvyrazneni += 1
                    if p.find_all("b")[0].text.strip() == p.find_all("b")[1].text.strip(): # kvůli 1996ps_stenprot_021schuz_s021021.htm
                        prvni_tucne_je_druhe_tucne = True
                    for f in funkce:
                        if f == p.text.strip().split(' ')[0].strip().lower(): # kvůli 1948ns_stenprot_054schuz_s054003.htm: Podpředseda Valo: Ďalším prihláseným rečníkom je pán posl. Navračič. Prosím, aby sa ujal slova.
                            if zdroj == True:
                                print(f.upper())
                            zadna_zminka_o_funkci = False
                if (pocitadlo_zvyrazneni > 1) & (prvni_tucne_je_druhe_tucne == False) & (zadna_zminka_o_funkci == True):
                    if zdroj == True:
                        print("dvakrát, bez akce")
                    odstavce.append(p.text.strip())

                else:
                    if zdroj == True:
                        print('mluvčí (: + a/b)')

                    if ("resoluční návrh" in p.text.lower()) or ("pozměňovací návrh" in p.text.lower()) or ("doplňovací návrh" in p.text.lower()):
                        odstavce.append(p.text.strip())
                        # kvůli 1920ns_ps_stenprot_078schuz_s078006.htm

                    else:
                        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.text.split(':')[0].strip()
                        try:
                            projev['mluvci_id'] = p.find("a")['href']
                        except:
                            if zdroj == True:
                                print('nenalezeno id mluvčích')
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

            elif p.find("b") and (len(p.text.strip()) < 100) and (p.text.strip().endswith(':')):
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.find('b').text.replace(':','').strip()
    
            elif p.find('a'): ## častá chyba, viz testy: je odkaz, není dvojtečka. kód se tu zbytečně duplikuje, whatever
                if zdroj == True:
                    print('ODKAZ')
                if p.text.strip().startswith(p.find('a').text.strip()) and (p.find("a").get("id") != "_d") and(p.find("a").get("name") != "OLE_LINK1"):
                    if p.find("a").text.strip().startswith("Hlasování"):
                        odstavce.append(p.text.strip())

                    else:
                    
                        # p.find("a").get("name") != "OLE_LINK1" kvůli 1946uns_stenprot_087schuz_s087005.htm
                        
                        if zdroj == True:
                            print('mluvčí p.find(a)')
        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.find("a").text.strip()
                        try:
                            projev['mluvci_id'] = p.find("a")['href']
                        except:
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

                else:
                    odstavce.append(p.text.strip())
                    
            else:
                odstavce.append(p.text.strip())
        
    projev['text'] = ciste_odstavce(odstavce)
    projevy.append(projev)    
    
    return slovnik_metadata, projevy

In [21]:
def html2data(soubor):
    
    ciste = oscrapovani_stenozaznamu(soubor)
    soubor = soubor.replace('.htm','')
    
    with open(os.path.join(kam, f'prepis_{soubor}.ndjson'), "w+", encoding="utf-8") as export:
        for projev in ciste[1]:
            export.write(json.dumps(projev) + '\n')

    with open(os.path.join(kam, f'meta_{soubor}.csv'), "w+", newline="", encoding="utf-8") as export_file:
        fieldnames = ciste[0].keys()
        writer = csv.DictWriter(export_file, fieldnames=fieldnames, quoting=csv.QUOTE_STRINGS)
        writer.writeheader()
        writer.writerow(ciste[0])

In [22]:
# oscrapovani_stenozaznamu("2021ps_stenprot_139schuz_b18300000.htm", zdroj=True)

In [23]:
oscrapovani_stenozaznamu("1998ps_stenprot_019schuz_s019364.htm", zdroj=True)

____
____ 
____(16.00 hodin)
____ Místopředseda PSP Stanislav Gross: Děkuji. Hlásí se ještě někdo do podrobné rozpravy? Ano. Pan kolega Janeček. 
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslanec Josef Janeček: Mně je skutečně líto, že jsme klesli k takovéto politické demagogii. Musím říci, že jsme navrhovali návrh
TUČNÉ
mluvčí (: + a/b)
____ 
____ Místopředseda PSP Stanislav Gross: Hlásí se ještě někdo do podrobné rozpravy? Reagovat bude ještě paní kolegyně Jirousová. 
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslankyně Taťána Jirousová: Vážený pane předsedající, já to nepovažuji, to, co jsem řekla, za demagogii, ale považuji spíše vyjá
TUČNÉ
mluvčí (: + a/b)
____ Já si myslím, že v určité věci se mohou shodnou pravicové i levicové strany. Já jsem to nemyslela demagogicky, ale myslela jsem t
____ 
____ Místopředseda PSP Stanislav Gross: Děkuji. Do podrobné rozpravy se již nikdo nehlásí, podrobnou rozpravu uzavírám. Táži se pana m
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslankyně Eva Fischerová: V rozpra

({'komora': 'ps',
  'obdobi': 1998,
  'schuze': 19,
  'cast': 364,
  'soubor': '1998ps_stenprot_019schuz_s019364.htm',
  'datum': '1999-12-21',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 1998 – 2002',
  'schuze_komplet': '19. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci_id': None,
   'text': '(16.00 hodin)'},
  {'poradi': 2,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci': 'Místopředseda PSP Stanislav Gross',
   'mluvci_id': '/sqw/p.sqw?P=460',
   'text': 'Děkuji. Hlásí se ještě někdo do podrobné rozpravy? Ano. Pan kolega Janeček.'},
  {'poradi': 3,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci': 'Poslanec Josef Janeček',
   'mluvci_id': '/sqw/p.sqw?P=480',
   'text': 'Mně je skutečně líto, že jsme klesli k takovéto politické demagogii. Musím říci, že jsme navrhovali návrh zákona o síti zdravotnických zařízení. Je tomu u

In [24]:
oscrapovani_stenozaznamu("1929ns_se_stenprot_197schuz_s197009.htm", zdroj=True)

____
____Místopředseda Donát (zroní): Dále má slovo pan sen. Beutel.
TUČNÉ
počítám tučnost
MÍSTOPŘEDSEDA
mluvčí (: + a/b)
nenalezeno id mluvčích
____
____Sen. Beutel (německy): Slavný senáte! Projednávání rozpočtu v poslanecké sněmovně a také zde v senátě poskytlo nám pestrý obraz rů
TUČNÉ
mluvčí (: + a/b)
nenalezeno id mluvčích
____
____Při této příležitosti dlužno konstatovati, že zajisté nejen tato okolnost, nýbrž především také bankovní hospodářství dřívější dob
____
____Pan ministerský předseda slíbil ve svém exposé změny v této příčině, především brzké snížení úrokové míry a opatření na snížení ce
____
____Hospodářská krise v jejím dosud nebývalém rozměru, hromadná nezaměstnanost s ciframi mocně stoupajícími, které dosud stále nedosáh
____
____Od té doby nabylo učiněno nic přiměřeného, poněvadž agrárníci jsou o tom jiného názoru. My sociální demokraté tedy na tom viny nem
TUČNÉ
____
____Slyšeli jsme teprve dnes zase od kol. Feierfeila v této příčině, že si přejí katolické školy. 

({'komora': 'se',
  'obdobi': 1929,
  'schuze': 197,
  'cast': 9,
  'soubor': '1929ns_se_stenprot_197schuz_s197009.htm',
  'datum': '1933-02-23',
  'komora_komplet': 'NS RČS 1929-1935, Senát',
  'schuze_komplet': '197. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'poradi': 1,
   'soubor': '1929ns_se_stenprot_197schuz_s197009.htm',
   'mluvci': 'Místopředseda Donát (zroní)',
   'mluvci_id': None,
   'text': 'Dále má slovo pan sen. Beutel.'},
  {'poradi': 2,
   'soubor': '1929ns_se_stenprot_197schuz_s197009.htm',
   'mluvci': 'Sen. Beutel (německy)',
   'mluvci_id': None,
   'text': 'Slavný senáte! Projednávání rozpočtu v poslanecké sněmovně a také zde v senátě poskytlo nám pestrý obraz růzností mínění o státním rozpočtu 1933. Když nyní konstatujeme, že výdaje jsou preliminovány 8632 mil. Kč a příjmy 8634 mil. Kč a oproti loňskému roku jsou o 2000 mil. Kč nižší, nutno konstatovati, že zde byla snaha spořiti; sedmičlenný výbor, k tomu účelu dosazený, vykonal skutečně, co bylo 

In [25]:
oscrapovani_stenozaznamu("2021ps_stenprot_108schuz_s108105.htm", zdroj=True)

____
____ 
TUČNÉ
____ 
____Místopředseda PSP Jan Bartošek: Antonín Gondolán. 
TUČNÉ
mluvčí (: + a/b)
____ 
____Místopředseda PSP Jan Skopeček: Zahajuji hlasování a ptám se, kdo je pro? Ať stiskne tlačítko a zdvihne ruku. Kdo je proti? 
TUČNÉ
mluvčí (: + a/b)
____Hlasování číslo 174, bylo přihlášeno 179 poslanců, pro hlasovalo 156, proti 1. Návrh byl přijat. 
ODKAZ
____ 
____Místopředseda PSP Jan Bartošek: Libor Grubhoffer. 
TUČNÉ
mluvčí (: + a/b)
____ 
____Místopředseda PSP Jan Skopeček: Zahajuji hlasování a ptám se, kdo je pro? Ať stiskne tlačítko a zdvihne ruku. Kdo je proti? 
TUČNÉ
mluvčí (: + a/b)
____Hlasování číslo 175, bylo přihlášeno 179 poslanců, pro hlasovalo 147, proti nebyl nikdo. Návrh byl přijat. 
ODKAZ
____ 
____Místopředseda PSP Jan Bartošek: Jan Gruntorád. 
TUČNÉ
mluvčí (: + a/b)
____ 
____Místopředseda PSP Jan Skopeček: Zahajuji hlasování a ptám se, kdo je pro? Ať stiskne tlačítko a zdvihne ruku. Kdo je proti? 
TUČNÉ
mluvčí (: + a/b)
____Hlasování číslo 176, bylo přih

({'komora': 'ps',
  'obdobi': 2021,
  'schuze': 108,
  'cast': 105,
  'soubor': '2021ps_stenprot_108schuz_s108105.htm',
  'datum': '2024-06-27',
  'komora_komplet': 'PČR, PS 2021-...',
  'schuze_komplet': '108. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'poradi': 1,
   'soubor': '2021ps_stenprot_108schuz_s108105.htm',
   'mluvci': 'Místopředseda PSP Jan Bartošek',
   'mluvci_id': '/sqw/detail.sqw?id=6178',
   'text': 'Antonín Gondolán.'},
  {'poradi': 2,
   'soubor': '2021ps_stenprot_108schuz_s108105.htm',
   'mluvci': 'Místopředseda PSP Jan Skopeček',
   'mluvci_id': '/sqw/detail.sqw?id=6418',
   'text': 'Zahajuji hlasování a ptám se, kdo je pro? Ať stiskne tlačítko a zdvihne ruku. Kdo je proti? Hlasování číslo 174, bylo přihlášeno 179 poslanců, pro hlasovalo 156, proti 1. Návrh byl přijat.'},
  {'poradi': 3,
   'soubor': '2021ps_stenprot_108schuz_s108105.htm',
   'mluvci': 'Místopředseda PSP Jan Bartošek',
   'mluvci_id': '/sqw/detail.sqw?id=6178',
   'text': 'Libor Gru

In [26]:
oscrapovani_stenozaznamu("2002ps_stenprot_035schuz_s035016.htm", zdroj=True)

____
____ 
____(15.30 hodin) (pokračuje Kalousek)
____ Článek 1. Zákon 361/2003 Sb., o služebním poměru příslušníků ozbrojených sborů, se mění takto: § 231 odstavec 1 zní: Tento zákon 
____ Článek 2. Zákon 362/2003 Sb., o změně zákonů souvisejících s přijetím zákona o služebním poměru příslušníků bezpečnostních sborů,
____ Tolik celkový pozměňující návrh v rámci podrobné rozpravy. Děkuji za pozornost.
____ 
____ Místopředseda PSP Jan Kasal: Děkuji. Rozumíme tomu, pane poslanče, tak, že ten vámi předešle zmiňovaný návrh se skartuje. K hlaso
TUČNÉ
mluvčí (: + a/b)
____ Dámy a pánové, zdá se, že další přihlášky do podrobné rozpravy nejsou, takže i podrobnou rozpravu končím. 
____ Odhlašuji vás, prosím o novou registraci. Budeme hlasovat o návrhu pana místopředsedy vlády, který navrhl, aby se lhůta pro třetí
____ Pro ty, kteří přišli, opakuji, že budeme hlasovat o návrhu pana navrhovatele, ministra práce a sociálních věcí, aby lhůta pro tře
____ 
____ Zahájil jsem hlasování. Kdo s tímto ná

({'komora': 'ps',
  'obdobi': 2002,
  'schuze': 35,
  'cast': 16,
  'soubor': '2002ps_stenprot_035schuz_s035016.htm',
  'datum': '2004-09-21',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 2002 – 2006',
  'schuze_komplet': '35. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2002ps_stenprot_035schuz_s035016.htm',
   'mluvci_id': None,
   'text': '(15.30 hodin) (pokračuje Kalousek) Článek 1. Zákon 361/2003 Sb., o služebním poměru příslušníků ozbrojených sborů, se mění takto: § 231 odstavec 1 zní: Tento zákon nabývá účinnosti dnem 1. ledna 2007. Článek 2. Zákon 362/2003 Sb., o změně zákonů souvisejících s přijetím zákona o služebním poměru příslušníků bezpečnostních sborů, se mění takto: Článek XVII. Tento zákon nabývá účinnosti dnem 1. ledna 2007. Tolik celkový pozměňující návrh v rámci podrobné rozpravy. Děkuji za pozornost.'},
  {'poradi': 2,
   'soubor': '2002ps_stenprot_035schuz_s035016.htm',
   'mluvci': '

In [27]:
oscrapovani_stenozaznamu("2010ps_stenprot_054schuz_s054069.htm", zdroj=True)

____
____ 
TUČNÉ
____(Jednání pokračovalo v 11.02 hodin.)
____ 
____Předsedkyně PSP Miroslava Němcová  Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci v
TUČNÉ
ODKAZ
mluvčí p.find(a)
____ 
____Budeme moci zahájit projednávání bodu číslo 111. Je to 
____ 
____111.  Vládní návrh, kterým se předkládá Parlamentu České republiky  k vyslovení souhlasu s ratifikací Protokol proti pašování přis
TUČNÉ
ODKAZ
____Prosím, aby se slova ujal ministr vnitra Jan Kubice. Prosím, pane ministře. (Slovo požaduje poslanec Filip pro faktickou poznámku.
____Pardon, pan kolega - faktická poznámka může být vznesena pouze v průběhu rozpravy, a rozprava se nekoná. (Poslanec Filip uplatňuje
____ 
____Poslanec Vojtěch Filip: Děkuji, paní předsedkyně. Já jsem rád, že tady máme pana ministra vnitra, ale nemyslím si, že je možné se 
TUČNÉ
mluvčí (: + a/b)
____ 
____Předsedkyně PSP Miroslava Němcová  Děkuji. Máme tedy dvě možnosti. Já se musím zeptat pana ministra, zda je o

({'komora': 'ps',
  'obdobi': 2010,
  'schuze': 54,
  'cast': 69,
  'soubor': '2010ps_stenprot_054schuz_s054069.htm',
  'datum': '2013-06-13',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 2010 – 2013',
  'schuze_komplet': '54. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci_id': None,
   'text': '(Jednání pokračovalo v 11.02 hodin.)'},
  {'poradi': 2,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci': 'Předsedkyně PSP Miroslava Němcová',
   'mluvci_id': '/sqw/detail.sqw?id=401',
   'text': 'Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci vás seznámit s omluvami, které mi byly ještě doručeny. Obdržela jsem omluvu pana ministra financí Miroslava Kalouska, který se omlouvá z jednání dnes odpoledne. Pak tady mám upozornění, že se omlouvá také pan ministr Blažek. Ano, pan ministr Blažek. To byly omluvy, k

In [28]:
oscrapovani_stenozaznamu("2021ps_stenprot_108schuz_s108004.htm", zdroj=True)

____
____ 
TUČNÉ
____ Mimochodem, to, co předvádí Fialova vláda a to, co se zase událo za těch několik dní nebo za ten krátký časový úsek, co tady ve S
____To všechno je samozřejmě pro hnutí SPD zcela nepřijatelné. My proto navrhujeme naopak zvýšit podporu všech občanů, kteří se zodpov
____Další záležitostí je, že teď podle aktuálních čísel se reálné mzdy za Fialovy vlády propadly téměř o 20 %. Přesto Fialova vláda pl
____My jako SPD v této souvislosti - a jsou to jasná čísla - navrhujeme, aby se v té první stupnici zvýšily platy o 15 %, což je v pod
____Podle našich kvalifikovaných propočtů by se zaměstnancům státu zařazeným v první stupnici, jak už jsem říkal, měly ty základní pla
____Jak jsem říkal, tak vy sami jste si tady vloni, Fialova vládní koalice, přidali platy politikům. My jsme hlasovali proti. Sami sob
____Ještě zmíním, že Fialova vláda opravdu likviduje všechny možné segmenty našich občanů, protože i třeba když se podíváme na zeměděl
____Tady u toho je pozoruhodné, že prá

({'komora': 'ps',
  'obdobi': 2021,
  'schuze': 108,
  'cast': 4,
  'soubor': '2021ps_stenprot_108schuz_s108004.htm',
  'datum': '2024-06-25',
  'komora_komplet': 'PČR, PS 2021-...',
  'schuze_komplet': '108. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2021ps_stenprot_108schuz_s108004.htm',
   'mluvci_id': None,
   'text': 'Mimochodem, to, co předvádí Fialova vláda a to, co se zase událo za těch několik dní nebo za ten krátký časový úsek, co tady ve Sněmovně naposled byla řádná schůze, tak teď od 1. července Fialova vláda se rozhodla opět poškodit současné i budoucí důchodce zhoršením podmínek penzijního spoření. SPD naopak prosazuje zvýšení státní podpory na důchodové spoření. Od 1. července se na návrh Fialovy vlády mění podmínky státní podpory penzijního spoření. Takže například lidem, kteří by chtěli penzijní spoření v důsledku zhoršení jeho podmínek ukončit předčasně, dokonce hrozí ztráta získaných státních příspěvků. Jiný

In [49]:
oscrapovani_stenozaznamu("1998ps_stenprot_019schuz_s019364.htm", zdroj=True)

____
____ 
____(16.00 hodin)
____ Místopředseda PSP Stanislav Gross: Děkuji. Hlásí se ještě někdo do podrobné rozpravy? Ano. Pan kolega Janeček. 
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslanec Josef Janeček: Mně je skutečně líto, že jsme klesli k takovéto politické demagogii. Musím říci, že jsme navrhovali návrh
TUČNÉ
mluvčí (: + a/b)
____ 
____ Místopředseda PSP Stanislav Gross: Hlásí se ještě někdo do podrobné rozpravy? Reagovat bude ještě paní kolegyně Jirousová. 
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslankyně Taťána Jirousová: Vážený pane předsedající, já to nepovažuji, to, co jsem řekla, za demagogii, ale považuji spíše vyjá
TUČNÉ
mluvčí (: + a/b)
____ Já si myslím, že v určité věci se mohou shodnou pravicové i levicové strany. Já jsem to nemyslela demagogicky, ale myslela jsem t
____ 
____ Místopředseda PSP Stanislav Gross: Děkuji. Do podrobné rozpravy se již nikdo nehlásí, podrobnou rozpravu uzavírám. Táži se pana m
TUČNÉ
mluvčí (: + a/b)
____ 
____ Poslankyně Eva Fischerová: V rozpra

({'komora': 'ps',
  'obdobi': 1998,
  'schuze': 19,
  'cast': 364,
  'soubor': '1998ps_stenprot_019schuz_s019364.htm',
  'datum': '1999-12-21',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 1998 – 2002',
  'schuze_komplet': '19. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci_id': None,
   'text': '(16.00 hodin)'},
  {'poradi': 2,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci': 'Místopředseda PSP Stanislav Gross',
   'mluvci_id': '/sqw/p.sqw?P=460',
   'text': 'Děkuji. Hlásí se ještě někdo do podrobné rozpravy? Ano. Pan kolega Janeček.'},
  {'poradi': 3,
   'soubor': '1998ps_stenprot_019schuz_s019364.htm',
   'mluvci': 'Poslanec Josef Janeček',
   'mluvci_id': '/sqw/p.sqw?P=480',
   'text': 'Mně je skutečně líto, že jsme klesli k takovéto politické demagogii. Musím říci, že jsme navrhovali návrh zákona o síti zdravotnických zařízení. Je tomu u

In [30]:
def process_file(soubor):
    try:
        html2data(soubor)
    except Exception as e:
        print(f"{soubor}: {e}")

In [31]:
directory = "downloads/schuze"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
# files = [f for f in files if "199" in f]
# files = [f for f in files if ('1992fs_sl' in f) or ('1990fs_sl' in f)] 
total_files = len(files)

batch_size = 8

# Process files in batches to reduce memory usage and improve progress tracking
for i in range(0, total_files, batch_size):
    batch = files[i:i + batch_size]
    
    # Use ProcessPoolExecutor for CPU-bound tasks or ThreadPoolExecutor for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map is more memory efficient than list comprehension for large batches
        futures = {executor.submit(process_file, f): f for f in batch}
        
        for j, future in enumerate(concurrent.futures.as_completed(futures), 1):
            try:
                future.result()  # Get the result to raise any exceptions
            except Exception as e:
                print(f"Error processing file {futures[future]}: {e}")
            
            # Update progress every batch_size files
            if (i + j) % batch_size == 0:
                print(f"{i + j}/{total_files}")

8/103255
16/103255
24/103255
32/103255
40/103255
48/103255
56/103255
64/103255
72/103255
80/103255
88/103255
96/103255
104/103255
112/103255
120/103255
128/103255
136/103255
144/103255
152/103255
160/103255
168/103255
176/103255
184/103255
192/103255
200/103255
208/103255
216/103255
224/103255
232/103255
240/103255
248/103255
256/103255
264/103255
272/103255
280/103255
288/103255
296/103255
304/103255
312/103255
320/103255
328/103255
336/103255
344/103255
352/103255
360/103255
368/103255
376/103255
384/103255
392/103255
400/103255
408/103255
416/103255
424/103255
432/103255
440/103255
448/103255
456/103255
464/103255
472/103255
480/103255
488/103255
496/103255
504/103255
512/103255
520/103255
528/103255
536/103255
544/103255
552/103255
560/103255
568/103255
576/103255
584/103255
592/103255
600/103255
608/103255
616/103255
624/103255
632/103255
640/103255
648/103255
656/103255
664/103255
672/103255
680/103255
688/103255
696/103255
704/103255
712/103255
720/103255
728/103255
736/103255
7